# STAT 857 - W23 Project 2
## Evan Callaghan | April 17, 2023

### 1. Configuring setup
Installing packages and loading libraries

In [ ]:
pip install lightgbm xgboost haversine optuna

In [110]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import math
import optuna
from haversine import haversine
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import RFECV, RFE
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier

pd.set_option('display.max_columns', None, 'display.max_rows', None)

### 2. Data Exploration Section

Reading the competition data files, exploring the training data, creating data visualizations 

In [111]:
## Reading the data
train = pd.read_csv('Data/W23P2_train.csv')
test = pd.read_csv('Data/W23P2_test.csv')
sub = pd.read_csv('Data/Sample_Submission.csv')

In [112]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
0,medium,1.0,4,40.7831,-73.9449,3675,1962 First Avenue,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,medium,1.0,3,40.7542,-73.9724,3600,155 East 47th Street,1,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,medium,1.0,2,40.7285,-73.9790,2900,206 Avenue B,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,medium,2.0,4,40.7306,-73.9837,7200,352 E 13 St,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,high,1.0,1,40.7645,-73.9840,2500,260 W 54 St.,0,0,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [113]:
test.head()

,ID,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
0,1,1.0,0,40.7769,-73.9467,1945,1661 York Avenue,0,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1.0,0,40.7610,-73.9990,2400,620 West 42nd Street,0,0,1,1,1,0,0,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1.0,2,40.7528,-73.9709,3750,235 E 46 Street,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1.0,3,40.7360,-73.9860,4450,170 East 18th Street,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,1.0,2,40.7084,-74.0048,5425,Cliff Street,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [114]:
print(train.shape)
print(test.shape)

(9846, 170)
(6564, 170)


In [115]:
train['interest_level'].value_counts() / train.shape[0]

low       0.394475
medium    0.377412
high      0.228113
Name: interest_level, dtype: float64

In [116]:
train.describe()

,bathrooms,bedrooms,latitude,longitude,price,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
count,9846.000000,9846.000000,9846.000000,9846.000000,9.846000e+03,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.0,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.

In [117]:
test.describe()

,ID,bathrooms,bedrooms,latitude,longitude,price,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
count,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.0,6564.0,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.0,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.0,6564.000000,6564.000000,6564.000000,6564.000000,6564.0,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000,6564.000000

### 3. Data Cleaning
Removing some outlier observations and condensing redundant variables

In [118]:
## Removing training observations with prices higher than $40,000
train = train[train['price'] < 40000].reset_index(drop = True)

## Removing outlier locations
train = train[(train['latitude'] < 42) & (train['latitude'] > 40)].reset_index(drop = True)

## Log transformation on the price variable
#train['price'] = np.log(train['price'])
#test['price'] = np.log(test['price'])

In [119]:
## Condensing redundant variables

laundry_vars = ['Laundry.in.Building', 'Laundry.in.Unit', 'Laundry.In.Building', 'Laundry.In.Unit', 'LAUNDRY', 'Washer.in.Unit', 
                'Dryer.in.Unit', 'Laundry.Room', 'Laundry', 'On.site.laundry', 'On.site.Laundry', 'Washer.Dryer', 'Washer.Dryer.in.building', 
                'In.Unit.Washer.Dryer', 'Washer...Dryer', 'Washer.Dryer.in.Unit']
parking_vars = ['Parking.Space', 'Garage', 'Parking', 'On.site.Garage', 'assigned.parking.space', 'Common.parking.Garage', 'Full.Service.Garage', 
               'On.site.Parking.Lot', 'Private.parking']
valet_vars = ['Valet.Parking', 'Valet']
deck_vars = ['Roof.Deck', 'Balcony', 'Terrace', 'Patio', 'Roof.deck', 'balcony', 'terrace', 'patio', 'private.balcony', 'Private.balcony', 
             'Private.Deck', 'Common.roof.deck', 'ROOFDECK']
outdoor_vars = ['Courtyard', 'Outdoor.Entertainment.Space', 'Private.Outdoor.Space', 'private.outdoor.space', 'Private.outdoor.space', 
                'Common.Outdoor.Space', 'PublicOutdoor', 'Outdoor.Space', 'Outdoor.Areas', 'Common.backyard', 'building.common.outdoor.space']
garden_vars = ['Common.garden', 'garden', 'Garden.Patio', 'Garden', 'Residents.Garden']
dishwasher_vars = ['Dishwasher', 'dishwasher']
gym_vars = ['Fitness.Center', 'Gym.Fitness', 'Health.Club', 'Gym', 'gym', 'Gym.In.Building']
pool_vars = ['Swimming.Pool', 'Pool', 'pool', 'Indoor.Pool']
elevator_vars = ['Elevator', 'elevator']
storage_vars = ['Storage', 'storage', 'Basement.Storage']
internet_vars = ['High.Speed.Internet', 'WiFi', 'WiFi.Access']
bike_vars = ['Bike.room', 'Bike.Room']
pet_friendly_vars = ['Dogs.Allowed', 'Cats.Allowed', 'Pet.Friendly', 'Pets.on.approval']
concierge_vars = ['Concierge', 'Concierge.Service', 'X24.7.Concierge']
doorman_vars = ['Doorman', 'Full.time.doorman', 'Virtual.Doorman', 'FT.Doorman', 'doorman']
super_vars = ['LIVE.IN.SUPER', 'Live.in.superintendent', 'Live.In.Superintendent','Live.in.Super', 'Live.In.Super']
hardwood_vars = ['Hardwood.Floors', 'HARDWOOD', 'Hardwood.floors', 'Hardwood']
ceiling_vars = ['High.ceilings', 'High.Ceilings', 'HIGH.CEILINGS', 'High.Ceiling']
brick_vars = ['EXPOSED.BRICK', 'Exposed.Brick']
construction_vars = ['New.Construction', 'Newly.renovated','Renovated', 'renovated', 'New.construction']
photo_vars = ['Actual.Apt..Photos', 'ACTUAL.APT..PHOTOS']
lounge_vars = ['Residents.Lounge', 'Lounge.room', 'Lounge']
playroom_vars = ['Childrens.Playroom', 'Children.s.Playroom']
ac_vars = ['Central.A.C', 'Air.conditioning']
kitchen_vars = ['EAT.IN.KITCHEN','Eat.In.Kitchen']
no_fee_vars = ['No.Fee', 'NO.FEE']
accessibity_vars = ['Wheelchair.Ramp', 'Wheelchair.Access']
multi_level_vars = ['Multi.Level', 'Multi.level']
fireplace_vars = ['Fireplace', 'Decorative.Fireplace']
highrise_vars = ['Hi.Rise', 'HIGHRISE']
marble_bath_vars = ['Marble.Bath', 'Marble.Bathroom']
prewar_vars = ['Pre.War', 'prewar', 'Prewar']

## Condensing training data
train['Has_Laundry'] = np.where(np.sum(train[laundry_vars], axis = 1) > 0, 1, 0)
train['Has_Parking'] = np.where(np.sum(train[parking_vars], axis = 1) > 0, 1, 0)
train['Has_Valet'] = np.where(np.sum(train[valet_vars], axis = 1) > 0, 1, 0)
train['Has_Deck'] = np.where(np.sum(train[deck_vars], axis = 1) > 0, 1, 0)
train['Has_Outdoor_Area'] = np.where(np.sum(train[outdoor_vars], axis = 1) > 0, 1, 0)
train['Has_Garden'] = np.where(np.sum(train[garden_vars], axis = 1) > 0, 1, 0)
train['Has_Dishwasher'] = np.where(np.sum(train[dishwasher_vars], axis = 1) > 0, 1, 0)
train['Has_Gym'] = np.where(np.sum(train[gym_vars], axis = 1) > 0, 1, 0)
train['Has_Pool'] = np.where(np.sum(train[pool_vars], axis = 1) > 0, 1, 0)
train['Has_Elevator'] = np.where(np.sum(train[elevator_vars], axis = 1) > 0, 1, 0)
train['Has_Storage'] = np.where(np.sum(train[storage_vars], axis = 1) > 0, 1, 0)
train['Has_Wifi'] = np.where(np.sum(train[internet_vars], axis = 1) > 0, 1, 0)
train['Has_Bike'] = np.where(np.sum(train[bike_vars], axis = 1) > 0, 1, 0)
train['Pet_Friendly'] = np.where(np.sum(train[pet_friendly_vars], axis = 1) > 0, 1, 0)
train['Has_Concierge'] = np.where(np.sum(train[concierge_vars], axis = 1) > 0, 1, 0)
train['Has_Doorman'] = np.where(np.sum(train[doorman_vars], axis = 1) > 0, 1, 0)
train['Has_Super'] = np.where(np.sum(train[super_vars], axis = 1) > 0, 1, 0)
train['Has_Hardwood_Floor'] = np.where(np.sum(train[hardwood_vars], axis = 1) > 0, 1, 0)
train['Has_High_Ceilings'] = np.where(np.sum(train[ceiling_vars], axis = 1) > 0, 1, 0)
train['Has_Brick'] = np.where(np.sum(train[brick_vars], axis = 1) > 0, 1, 0)
train['Has_Renovated'] = np.where(np.sum(train[construction_vars], axis = 1) > 0, 1, 0)
train['Has_Photos'] = np.where(np.sum(train[photo_vars], axis = 1) > 0, 1, 0)
train['Has_Lounge'] = np.where(np.sum(train[lounge_vars], axis = 1) > 0, 1, 0)
train['Has_Playroom'] = np.where(np.sum(train[playroom_vars], axis = 1) > 0, 1, 0)
train['Has_AC'] = np.where(np.sum(train[ac_vars], axis = 1) > 0, 1, 0)
train['Has_Kitchen'] = np.where(np.sum(train[kitchen_vars], axis = 1) > 0, 1, 0)
train['No_Fee'] = np.where(np.sum(train[no_fee_vars], axis = 1) > 0, 1, 0)
train['Accessible'] = np.where(np.sum(train[accessibity_vars], axis = 1) > 0, 1, 0)
train['Multi_Level'] = np.where(np.sum(train[multi_level_vars], axis = 1) > 0, 1, 0)
train['Fire'] = np.where(np.sum(train[fireplace_vars], axis = 1) > 0, 1, 0)
train['Highrise'] = np.where(np.sum(train[highrise_vars], axis = 1) > 0, 1, 0)
train['Marble_Bath'] = np.where(np.sum(train[marble_bath_vars], axis = 1) > 0, 1, 0)
train['Pre_War'] = np.where(np.sum(train[prewar_vars], axis = 1) > 0, 1, 0)

## Condensing testing data
test['Has_Laundry'] = np.where(np.sum(test[laundry_vars], axis = 1) > 0, 1, 0)
test['Has_Parking'] = np.where(np.sum(test[parking_vars], axis = 1) > 0, 1, 0)
test['Has_Valet'] = np.where(np.sum(test[valet_vars], axis = 1) > 0, 1, 0)
test['Has_Deck'] = np.where(np.sum(test[deck_vars], axis = 1) > 0, 1, 0)
test['Has_Outdoor_Area'] = np.where(np.sum(test[outdoor_vars], axis = 1) > 0, 1, 0)
test['Has_Garden'] = np.where(np.sum(test[garden_vars], axis = 1) > 0, 1, 0)
test['Has_Dishwasher'] = np.where(np.sum(test[dishwasher_vars], axis = 1) > 0, 1, 0)
test['Has_Gym'] = np.where(np.sum(test[gym_vars], axis = 1) > 0, 1, 0)
test['Has_Pool'] = np.where(np.sum(test[pool_vars], axis = 1) > 0, 1, 0)
test['Has_Elevator'] = np.where(np.sum(test[elevator_vars], axis = 1) > 0, 1, 0)
test['Has_Storage'] = np.where(np.sum(test[storage_vars], axis = 1) > 0, 1, 0)
test['Has_Wifi'] = np.where(np.sum(test[internet_vars], axis = 1) > 0, 1, 0)
test['Has_Bike'] = np.where(np.sum(test[bike_vars], axis = 1) > 0, 1, 0)
test['Pet_Friendly'] = np.where(np.sum(test[pet_friendly_vars], axis = 1) > 0, 1, 0)
test['Has_Concierge'] = np.where(np.sum(test[concierge_vars], axis = 1) > 0, 1, 0)
test['Has_Doorman'] = np.where(np.sum(test[doorman_vars], axis = 1) > 0, 1, 0)
test['Has_Super'] = np.where(np.sum(test[super_vars], axis = 1) > 0, 1, 0)
test['Has_Hardwood_Floor'] = np.where(np.sum(test[hardwood_vars], axis = 1) > 0, 1, 0)
test['Has_High_Ceilings'] = np.where(np.sum(test[ceiling_vars], axis = 1) > 0, 1, 0)
test['Has_Brick'] = np.where(np.sum(test[brick_vars], axis = 1) > 0, 1, 0)
test['Has_Renovated'] = np.where(np.sum(test[construction_vars], axis = 1) > 0, 1, 0)
test['Has_Photos'] = np.where(np.sum(test[photo_vars], axis = 1) > 0, 1, 0)
test['Has_Lounge'] = np.where(np.sum(test[lounge_vars], axis = 1) > 0, 1, 0)
test['Has_Playroom'] = np.where(np.sum(test[playroom_vars], axis = 1) > 0, 1, 0)
test['Has_AC'] = np.where(np.sum(test[ac_vars], axis = 1) > 0, 1, 0)
test['Has_Kitchen'] = np.where(np.sum(test[kitchen_vars], axis = 1) > 0, 1, 0)
test['No_Fee'] = np.where(np.sum(test[no_fee_vars], axis = 1) > 0, 1, 0)
test['Accessible'] = np.where(np.sum(test[accessibity_vars], axis = 1) > 0, 1, 0)
test['Multi_Level'] = np.where(np.sum(test[multi_level_vars], axis = 1) > 0, 1, 0)
test['Fire'] = np.where(np.sum(test[fireplace_vars], axis = 1) > 0, 1, 0)
test['Highrise'] = np.where(np.sum(test[highrise_vars], axis = 1) > 0, 1, 0)
test['Marble_Bath'] = np.where(np.sum(test[marble_bath_vars], axis = 1) > 0, 1, 0)
test['Pre_War'] = np.where(np.sum(test[prewar_vars], axis = 1) > 0, 1, 0)

to_drop = [laundry_vars, parking_vars, valet_vars, deck_vars, outdoor_vars,garden_vars, dishwasher_vars, gym_vars, pool_vars, elevator_vars, 
           storage_vars, internet_vars, bike_vars, pet_friendly_vars, concierge_vars, doorman_vars, super_vars, hardwood_vars, ceiling_vars, 
           brick_vars, construction_vars, photo_vars, lounge_vars, playroom_vars, ac_vars, kitchen_vars, no_fee_vars, accessibity_vars, 
           multi_level_vars, fireplace_vars, highrise_vars, marble_bath_vars, prewar_vars] 

for cols in to_drop:
    train = train.drop(columns = cols)
    test = test.drop(columns = cols)

In [120]:
## Removing variables which appear less than 25 times in both the training and testing sets
train_categorical = train.iloc[:, 7:]
test_categorical = test.iloc[:, 7:]

for col in train_categorical.columns:
    if ((train_categorical[col].sum() < 25) & (test_categorical[col].sum() < 25)):
        train = train.drop(columns = col)
        test = test.drop(columns = col)

### 4. Variable Engineering
Creating new variables based on property type and provided amenities, scaling the numerical input variables

In [124]:
## Combining data frames for feature engineering purposes
train['train'] = 1
train.insert(0, 'train', 1)
test['train'] = 0
full_data = pd.concat([train.drop(columns = ['interest_level']), test.drop(columns = ['ID'])])

In [125]:
full_data.head()

,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Reduced.Fee,Exclusive,LOWRISE,SIMPLEX,Furnished,Loft,Stainless.Steel.Appliances,Green.Building,Short.Term.Allowed,Subway,Granite.Kitchen,Light,Has_Laundry,Has_Parking,Has_Deck,Has_Outdoor_Area,Has_Garden,Has_Dishwasher,Has_Gym,Has_Pool,Has_Elevator,Has_Storage,Has_Wifi,Pet_Friendly,Has_Concierge,Has_Doorman,Has_Super,Has_Hardwood_Floor,Has_High_Ceilings,Has_Renovated,Has_Lounge,No_Fee,Accessible,Multi_Level,Fire,Marble_Bath,Pre_War,train
0,1.0,4,40.7831,-73.9449,3675,1962 First Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
1,1.0,3,40.7542,-73.9724,3600,155 East 47th Street,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1
2,1.0,2,40.7285,-73.9790,2900,206 Avenue B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
3,2.0,4,40.7306,-73.9837,7200,352 E 13 St,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1
4,1.0,1,40.7645,-73.9840,2500,260 W 54 St.,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1


In [106]:
## Adding geolocation features
full_data['geo_area_50'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:(int(x[0] * 50) % 50) * 50 + (int(-x[1] * 50) % 50), axis = 1)                                         
                         
full_data['geo_area_100'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:(int(x[0] * 100) % 100) * 100 + (int(-x[1] * 100) % 100), axis = 1)                                         
  
full_data['geo_area_200'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:(int(x[0] * 200) % 200) * 200 + (int(-x[1] * 200) % 200), axis = 1)                                         

## Financial district
lat = 40.705628
lon = -74.010278
full_data['distance_to_fin'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:math.sqrt((x[0] - lat)**2 + (x[1] - lon)**2), axis = 1)

## Central park
lat = 40.785091
lon = -73.968285
full_data['distance_to_cp'] = full_data[['latitude', 'longitude']]\
.apply(lambda x:math.sqrt((x[0] - lat)**2 + (x[1] - lon)**2), axis = 1)

## Stroing the variable names
geo_cat_vars = ['geo_area_50', 'geo_area_100', 'geo_area_200']
geo_num_vars = ['distance_to_fi', 'distance_to_cp']

In [ ]:
## Number of total rooms
full_data['rooms'] = full_data['bedrooms'] + full_data['bathrooms'] 

## Number of categorical features listed 
full_data['num_of_features'] = full_data['features'].apply(lambda x:len(x))

In [78]:
## Price per bedroom
rooms_train = train['bedrooms'].apply(lambda x: max(x, 0.5))
train['price_per_bedroom'] = train['price'] / rooms_train

rooms_test = test['bedrooms'].apply(lambda x: max(x, 0.5))
test['price_per_bedroom'] = test['price'] / rooms_test

In [79]:
## Standardizing numeric variables
def min_max_scaler(data, minimun, maximum):
    data_scaled = (data - minimun) / (maximum - minimun)
    return data_scaled

test['price'] = min_max_scaler(test['price'], np.min(train['price']), np.max(train['price']))
test['bathrooms'] = min_max_scaler(test['bathrooms'], np.min(train['bathrooms']), np.max(train['bathrooms']))
test['bedrooms'] = min_max_scaler(test['bedrooms'], np.min(train['bedrooms']), np.max(train['bedrooms']))
test['distance'] = min_max_scaler(test['distance'], np.min(train['distance']), np.max(train['distance']))
test['price_per_bedroom'] = min_max_scaler(test['price_per_bedroom'], np.min(train['price_per_bedroom']), np.max(train['price_per_bedroom']))
test['latitude'] = min_max_scaler(test['latitude'], np.min(train['latitude']), np.max(train['latitude']))
test['longitude'] = min_max_scaler(test['longitude'], np.min(train['longitude']), np.max(train['longitude']))

train['price'] = min_max_scaler(train['price'], np.min(train['price']), np.max(train['price']))
train['bathrooms'] = min_max_scaler(train['bathrooms'], np.min(train['bathrooms']), np.max(train['bathrooms']))
train['bedrooms'] = min_max_scaler(train['bedrooms'], np.min(train['bedrooms']), np.max(train['bedrooms']))
train['distance'] = min_max_scaler(train['distance'], np.min(train['distance']), np.max(train['distance']))
train['price_per_bedroom'] = min_max_scaler(train['price_per_bedroom'], np.min(train['price_per_bedroom']), np.max(train['price_per_bedroom']))
train['latitude'] = min_max_scaler(train['latitude'], np.min(train['latitude']), np.max(train['latitude']))
train['longitude'] = min_max_scaler(train['longitude'], np.min(train['longitude']), np.max(train['longitude']))

In [80]:
## Creating final data-frames
train = train.drop(columns = ['street_address'])
test = test.drop(columns = ['street_address'])

In [64]:
## Mean of each feature by interest level class
interests = pd.DataFrame(train.groupby('interest_level').mean())
interests.iloc[[1, 2, 0], :]

,bathrooms,bedrooms,latitude,longitude,price,Dining.Room,Reduced.Fee,Exclusive,LOWRISE,SIMPLEX,Furnished,Loft,Stainless.Steel.Appliances,Green.Building,Short.Term.Allowed,Subway,Granite.Kitchen,Light,Has_Laundry,Has_Parking,Has_Deck,Has_Outdoor_Area,Has_Garden,Has_Dishwasher,Has_Gym,Has_Pool,Has_Elevator,Has_Storage,Has_Wifi,Pet_Friendly,Has_Concierge,Has_Doorman,Has_Super,Has_Hardwood_Floor,Has_High_Ceilings,Has_Renovated,Has_Lounge,No_Fee,Accessible,Multi_Level,Fire,Marble_Bath,Pre_War,distance,price_per_bedroom
interest_level,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
low,0.222516,0.247466,0.655793,0.601274,0.477137,0.092010,0.006959,0.041753,0.018814,0.022938,0.007474,0.041495,0.001546,0.003866,0.000515,0.000515,0.000773,0.001031,0.454124,0.035052,0.191753,0.132732,0.037629,0.384278,0.281443,0.069588,0.544330,0.005670,0.081186,0.495876,0.006443,0.447423,0.011856,0.444072,0.011340,0.047938,0.006443,0.307990,0.024485,0.014691,0.021907,0.000773,0.233247,0.046920,0.350210
medium,0.212211,0.273127,0.651809,0.603049,0.442911,0.130552,0.026110,0.034455,0.005653,0.006460,0.023149,0.053567,0.003769,0.004307,0.001077,0.004576,0.005653,0.005922,0.572544,0.015612,0.239569,0.195962,0.055720,0.511978,0.266218,0.062719,0.552894,0.001346,0.118977,0.445491,0.001077,0.404038,0.007806,0.623957,0.019112,0.079946,0.001615,0.478062,0.034993,0.012113,0.013728,0.005653,0.190848,0.053043,0.311418
high,0.202389,0.252710,0.650707,0.609474,0.399837,0.109131,0.046325,0.073051,0.006236,0.006682,0.031626,0.042762,0.004454,0.001336,0.008463,0.004454,0.005345,0.005791,0.470379,0.019599,0.158575,0.175056,0.051670,0.385301,0.181737,0.039198,0.466370,0.004454,0.084187,0.424053,0.003118,0.278842,0.006236,0.517595,0.022717,0.062806,0.000445,0.450780,0.024053,0.012027,0.010690,0.004900,0.140757,0.065563,0.329187


In [65]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,Dining.Room,Reduced.Fee,Exclusive,LOWRISE,SIMPLEX,Furnished,Loft,Stainless.Steel.Appliances,Green.Building,Short.Term.Allowed,Subway,Granite.Kitchen,Light,Has_Laundry,Has_Parking,Has_Deck,Has_Outdoor_Area,Has_Garden,Has_Dishwasher,Has_Gym,Has_Pool,Has_Elevator,Has_Storage,Has_Wifi,Pet_Friendly,Has_Concierge,Has_Doorman,Has_Super,Has_Hardwood_Floor,Has_High_Ceilings,Has_Renovated,Has_Lounge,No_Fee,Accessible,Multi_Level,Fire,Marble_Bath,Pre_War,distance,price_per_bedroom
0,medium,0.181818,0.666667,0.687197,0.627404,0.489611,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0.015424,0.033537
1,medium,0.181818,0.500000,0.657431,0.602098,0.485054,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0.028949,0.072919
2,medium,0.181818,0.333333,0.630961,0.596025,0.437266,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.060472,0.145995
3,medium,0.363636,0.666667,0.633124,0.591700,0.638245,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0.058942,0.043314
4,high,0.181818,0.166667,0.668040,0.591424,0.404464,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0.021840,0.369169


In [47]:
test.head()

,ID,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Reduced.Fee,Exclusive,LOWRISE,SIMPLEX,Furnished,Loft,Stainless.Steel.Appliances,Green.Building,Short.Term.Allowed,Subway,Granite.Kitchen,Light,Has_Laundry,Has_Parking,Has_Deck,Has_Outdoor_Area,Has_Garden,Has_Dishwasher,Has_Gym,Has_Pool,Has_Elevator,Has_Storage,Has_Wifi,Pet_Friendly,Has_Concierge,Has_Doorman,Has_Super,Has_Hardwood_Floor,Has_High_Ceilings,Has_Renovated,Has_Lounge,No_Fee,Accessible,Multi_Level,Fire,Marble_Bath,Pre_War,distance,price_per_bedroom
0,1,0.181818,0.000000,0.680812,0.625748,0.348985,1661 York Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0.014381,0.794950
1,2,0.181818,0.000000,0.664435,0.577620,0.395442,620 West 42nd Street,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0.034567,0.819398
2,3,0.181818,0.333333,0.655989,0.603478,0.494076,235 E 46 Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0.030409,0.153469
3,4,0.181818,0.500000,0.638686,0.589583,0.531901,170 East 18th Street,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0.053307,0.077027
4,5,0.181818,0.333333,0.610259,0.572283,0.575686,Cliff Street,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.090437,0.164206


### 5. Splitting the data
Splitting the training data into separate training and validation sets for future modelling purposes, saving new files

In [58]:
## Defining input and target variables
X = train.drop(columns = ['interest_level'])
Y = train['interest_level']

## Splitting the data
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 42)

## Saving new csv files
pd.concat([X_train, Y_train], axis = 1).to_csv('Data/Training.csv', index = False)
pd.concat([X_validation, Y_validation], axis = 1).to_csv('Data/Validation.csv', index = False)
test.to_csv('Data/Testing.csv', index = False)

### 6. Feature Selection
Performing recursive feature elimination to determine which variables are the most influential

In [2]:
## Using RFE with RandomForestClassifer to identify most important features
def flat_list(my_list):
    
    ## Defining list to store results
    out_list = list()
    for i in my_list:
        out_list += i
    return out_list

def RF_RFE_rep_cross_val(X, Y, numb_folds, max_features, numb_reps):
    
    ## Defining list to store results
    RFE_rep_results = list()
    for i in range(2, max_features):
        RFE_rep_results.append(RF_rep_cross_val(X, Y, numb_folds, i, numb_reps))
        print('Features -->', i) ## Sanity check
    return RFE_rep_results

def RF_rep_cross_val(X, Y, numb_folds, numb_features, numb_reps):
    
    ## Defining the list to store results
    rep_results = list()
    for i in range(0, numb_reps):
        rep_results.append(RF_cross_val(X, Y, numb_folds, numb_features))
    return flat_list(rep_results)

def RF_cross_val(X, Y, numb_folds, numb_features):
    
    ## Defining list to store results
    results = list()
    
    ## Defining the number of folds
    kf = KFold(n_splits = numb_folds, shuffle = True)
    
    for train_index, test_index in kf.split(X):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        ## Running RFE with i features
        RF_rfe = RFE(estimator = RandomForestClassifier(n_estimators = 100, max_depth = 5), 
                     n_features_to_select = numb_features).fit(X_train, Y_train)
        
        ## Variables to be considered
        to_select = X_train.columns[RF_rfe.support_]
        to_select_list.append(RF_rfe.support_)
        
        ## Building the Random Forest model
        X_train_md = X_train[to_select]
        X_test_md = X_test[to_select]
        
        RF_md = RandomForestClassifier(n_estimators = 100, max_depth = 5).fit(X_train_md, Y_train)
        
        ## Predicting on the test data-frame and storing RMSE
        results.append(log_loss(Y_test, RF_md.predict_proba(X_test_md)))

    return results

## Defining list to store results
to_select_list = list()

## Defining input and target variables
data = pd.read_csv('Data/Training.csv')
X = data.drop(columns = ['interest_level']); Y = data['interest_level']

## Running RFE to estimate number of features to be selected
RFE_numb_features = RF_RFE_rep_cross_val(X, Y, numb_folds = 5, max_features = 31, numb_reps = 1)

Features --> 2
Features --> 3
Features --> 4
Features --> 5
Features --> 6
Features --> 7
Features --> 8
Features --> 9
Features --> 10
Features --> 11
Features --> 12
Features --> 13
Features --> 14
Features --> 15
Features --> 16
Features --> 17
Features --> 18
Features --> 19
Features --> 20
Features --> 21
Features --> 22
Features --> 23
Features --> 24
Features --> 25
Features --> 26
Features --> 27
Features --> 28
Features --> 29
Features --> 30


In [3]:
## Identifying features
features = pd.DataFrame(to_select_list)
features.columns = X.columns
feature_selections = 100 * features.apply(np.sum, axis = 0) / features.shape[0]
feature_selections = pd.DataFrame(feature_selections).reset_index(drop = False)

## Model performance given the number of variables
feature_performance = pd.DataFrame(RFE_numb_features)
feature_performance.columns = [['Split_1', 'Split_2', 'Split_3']]
feature_performance['Mean'] = feature_performance.apply(np.mean, axis = 1)
feature_performance['Num_features'] = feature_performance.index + 2

In [4]:
feature_performance

,Split_1,Split_2,Split_3,Mean,Num_features
0,0.949621,0.937591,0.955338,0.947516,2
1,0.926319,0.948261,0.940013,0.938198,3
2,0.933429,0.933055,0.919642,0.928709,4
3,0.931565,0.937130,0.926539,0.931745,5
4,0.928603,0.922935,0.930155,0.927231,6
5,0.933931,0.941083,0.931043,0.935352,7
6,0.928746,0.935144,0.947655,0.937182,8
7,0.924838,0.931834,0.928597,0.928423,9
8,0.929225,0.930006,0.935609,0.931613,10
9,0.948635,0.926157,0.919035,0.931276,11


In [5]:
feature_selections.sort_values(0, ascending = False).head(20)

,index,0
4,price,100.000000
5,price_per_bedroom,100.000000
34,Amenity_Low,96.551724
35,Amenity_Medium,91.954023
8,distance,89.655172
7,No_Fee,86.206897
2,latitude,81.609195
3,longitude,81.609195
33,Amenity_High,73.563218
27,Hardwood_Floor,71.264368


### 7. Hyper-Parameter Tuning
Tuning the hyper-parameters for Random Forest, Hist Gradient Boosting, LightGBM, and XGBoost using Optuna framework

In [12]:
## Reading the data
train = pd.read_csv('Data/Training.csv')
val = pd.read_csv('Data/Validation.csv')

## Defining the input and target variables
X_train = train.drop(columns = ['interest_level'])
X_validation = val.drop(columns = ['interest_level'])

Y_train = train['interest_level']
Y_validation = val['interest_level']

## Adjusting target variable
Y_train = np.where(Y_train == 'low', 0, np.where(Y_train == 'medium', 1, 2))
Y_validation = np.where(Y_validation == 'low', 0, np.where(Y_validation == 'medium', 1, 2))

In [17]:
def rf_objective(trial):

    ## Defining the XGBoost hyper-parameter grid
    rf_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 100),
                     'max_depth': trial.suggest_int('max_depth', 3, 12), 
                     'min_samples_split': trial.suggest_int('min_samples_split', 1, 300, 50), 
                     'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 300, 25), 
                     'random_state': trial.suggest_int('random_state', 1, 500),
                    }
    
    ## Building the model
    rf_md = RandomForestClassifier(**rf_param_grid, criterion = 'log_loss', n_jobs = -1).fit(X_train, Y_train)
    
    ## Predicting on the test data-frame
    rf_md_preds = rf_md.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    rf_md_error = log_loss(Y_validation, rf_md_preds)
    
    return rf_md_error

def xgb_objective(trial):

    ## Defining the XGBoost hyper-parameter grid
    xgboost_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step = 100), 
                          'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01), 
                          'max_depth': trial.suggest_int('max_depth', 3, 15), 
                          'gamma': trial.suggest_float('gamma', 0, 100, step = 10), 
                          'min_child_weight': trial.suggest_int('min_child_weight', 0, 300, step = 20), 
                          'subsample': trial.suggest_float('subsample', 0.6, 1, step = 0.05), 
                          'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1, step = 0.05),
                          'seed': trial.suggest_int('seed', 1, 1000)
                         }
    
    ## Building the model
    xgb_md = XGBClassifier(**xgboost_param_grid, n_jobs = -1).fit(X_train, Y_train)
    
    ## Predicting on the test data-frame
    xgb_md_preds = xgb_md.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    xgb_md_error = log_loss(Y_validation, xgb_md_preds)
    
    return xgb_md_error

def lgbm_objective(trial):
    
    ## Defining the LGB hyper-parameter grid
    LGB_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1500, step = 100),
                      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01),
                      'num_leaves': trial.suggest_int('num_leaves', 5, 40, step = 1),
                      'max_depth': trial.suggest_int('max_depth', 3, 12),
                      'subsample': trial.suggest_float('subsample', 0.7, 1, step = 0.01), 
                      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1, step = 0.01),
                      'random_state': trial.suggest_int('random_state', 1, 1000),
                      'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.5, step = 0.1),
                      'reg_lambda': trial.suggest_float('reg_lambda', 0, 0.5, step = 0.1), 
                      'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 0, 300, step = 20),
                      'objective': 'log_loss', 
                      'verbosity': -1
                     }
                     
    ## Building the LightGBM model
    model = LGBMClassifier(**LGB_param_grid, n_jobs = -1).fit(X_train, Y_train)
        
    ## Predicting on the test data-frame
    lgbm_md_preds = model.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    lgbm_md_error = log_loss(Y_validation, lgbm_md_preds)
    
    return lgbm_md_error

def hist_objective(trial):

    ## Defining the XGBoost hyper-parameter grid
    hist_param_grid = {'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01),
                       'max_iter': trial.suggest_int('max_iter', 100, 1000, 100),
                       'max_depth': trial.suggest_int('max_depth', 3, 15), 
                       'l2_regularization': trial.suggest_float('l2_regularization', 0, 0.5, step = 0.01),
                       'random_state': trial.suggest_int('random_state', 1, 500),
                      }
    
    ## Building the model
    hist_md = HistGradientBoostingClassifier(**hist_param_grid, loss = 'log_loss', early_stopping = True).fit(X_train, Y_train)
    
    ## Predicting on the test data-frame
    hist_md_preds = hist_md.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    hist_md_error = log_loss(Y_validation, hist_md_preds)
    
    return hist_md_error

In [20]:
## Starting RandomForest
## ----
## Creating a study object and to optimize the home objective function
study_rf = optuna.create_study(direction = 'minimize')
study_rf.optimize(rf_objective, n_trials = 500)

## Starting XGBoost
## ----
## Creating a study object and to optimize the home objective function
study_xgb = optuna.create_study(direction = 'minimize')
study_xgb.optimize(xgb_objective, n_trials = 500)

## Starting LightGBM
## ----
## Creating a study object and to optimize the home objective function
study_lgbm = optuna.create_study(direction = 'minimize')
study_lgbm.optimize(lgbm_objective, n_trials = 500)

## Starting HistGradientBoosting
## ----
## Creating a study object and to optimize the home objective function
study_hist = optuna.create_study(direction = 'minimize')
study_hist.optimize(hist_objective, n_trials = 500)

[I 2023-04-13 15:02:18,115] A new study created in memory with name: no-name-5f5b6deb-5950-4380-89d5-b21e7ca16de2
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [1, 300] and step=50, but the range is not divisible by `step`. It will be replaced by [1, 251].
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [1, 300] and step=25, but the range is not divisible by `step`. It will be replaced by [1, 276].
  warnings.warn(
[I 2023-04-13 15:02:21,372] Trial 0 finished with value: 0.9419347598004658 and parameters: {'n_estimators': 800, 'max_depth': 6, 'min_samples_split': 201, 'min_samples_leaf': 1, 'random_state': 412}. Best is trial 0 with value: 0.9419347598004658.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/distributions.py:716: UserWarning: The distribution

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100


[I 2023-04-13 16:08:39,039] Trial 0 finished with value: 1.2090174509717941 and parameters: {'n_estimators': 1200, 'learning_rate': 0.3, 'num_leaves': 16, 'max_depth': 6, 'subsample': 0.8999999999999999, 'colsample_bytree': 0.8999999999999999, 'random_state': 798, 'reg_alpha': 0.2, 'reg_lambda': 0.0, 'min_data_in_leaf': 100}. Best is trial 0 with value: 1.2090174509717941.


[LightGBM] [Warning] min_data_in_leaf is set=180, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=180


[I 2023-04-13 16:08:41,948] Trial 1 finished with value: 0.9794022797155288 and parameters: {'n_estimators': 800, 'learning_rate': 0.24000000000000002, 'num_leaves': 24, 'max_depth': 6, 'subsample': 0.73, 'colsample_bytree': 0.9099999999999999, 'random_state': 916, 'reg_alpha': 0.2, 'reg_lambda': 0.2, 'min_data_in_leaf': 180}. Best is trial 1 with value: 0.9794022797155288.


...

[I 2023-04-13 16:43:46,721] Trial 497 finished with value: 0.8319705008661703 and parameters: {'n_estimators': 500, 'learning_rate': 0.02, 'num_leaves': 14, 'max_depth': 7, 'subsample': 0.7799999999999999, 'colsample_bytree': 0.71, 'random_state': 808, 'reg_alpha': 0.30000000000000004, 'reg_lambda': 0.30000000000000004, 'min_data_in_leaf': 0}. Best is trial 297 with value: 0.8237743890254952.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:132: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:149: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


[I 2023-04-13 16:43:50,984] Trial 498 finished with value: 0.9070008646437021 and parameters: {'n_estimators': 1400, 'learning_rate': 0.05, 'num_leaves': 19, 'max_depth': 8, 'subsample': 0.85, 'colsample_bytree': 0.7, 'random_state': 890, 'reg_alpha': 0.30000000000000004, 'reg_lambda': 0.30000000000000004, 'min_data_in_leaf': 20}. Best is trial 297 with value: 0.8237743890254952.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:132: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/samplers/_tpe/_erf.py:149: RuntimeWarning: divide by zero encountered in true_divide
  s = one / (x * x)


[LightGBM] [Warning] min_data_in_leaf is set=0, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=0


[I 2023-04-13 16:43:55,269] Trial 499 finished with value: 0.84062435538999 and parameters: {'n_estimators': 1400, 'learning_rate': 0.03, 'num_leaves': 16, 'max_depth': 8, 'subsample': 0.86, 'colsample_bytree': 0.71, 'random_state': 773, 'reg_alpha': 0.30000000000000004, 'reg_lambda': 0.30000000000000004, 'min_data_in_leaf': 0}. Best is trial 297 with value: 0.8237743890254952.
[I 2023-04-13 16:43:55,270] A new study created in memory with name: no-name-9426c103-70be-4041-a77c-63fa2f12ba57
[I 2023-04-13 16:43:55,889] Trial 0 finished with value: 0.8529731135697965 and parameters: {'learning_rate': 0.19, 'max_iter': 100, 'max_depth': 8, 'l2_regularization': 0.4, 'random_state': 208}. Best is trial 0 with value: 0.8529731135697965.
[I 2023-04-13 16:43:57,045] Trial 1 finished with value: 0.8442620125244725 and parameters: {'learning_rate': 0.08, 'max_iter': 100, 'max_depth': 10, 'l2_regularization': 0.07, 'random_state': 138}. Best is trial 1 with value: 0.8442620125244725.
[I 2023-04-13

In [21]:
## Printing best hyper-parameter set
print('Random Forest: \n', study_rf.best_trial.params)
print(study_rf.best_trial.value)

## Printing best hyper-parameter set
print('\nHistGB: \n', study_hist.best_trial.params)
print(study_hist.best_trial.value)

## Printing best hyper-parameter set
print('\nXGBoost: \n', study_xgb.best_trial.params)
print(study_xgb.best_trial.value)

## Printing best hyper-parameter set
print('\nLightGBM: \n', study_lgbm.best_trial.params)
print(study_lgbm.best_trial.value)

Random Forest: 
 {'n_estimators': 500, 'max_depth': 12, 'min_samples_split': 1, 'min_samples_leaf': 1, 'random_state': 329}
0.8658326242014183

HistGB: 
 {'learning_rate': 0.02, 'max_iter': 300, 'max_depth': 12, 'l2_regularization': 0.33, 'random_state': 271}
0.8258689528641616

XGBoost: 
 {'n_estimators': 200, 'learning_rate': 0.22, 'max_depth': 3, 'gamma': 0.0, 'min_child_weight': 0, 'subsample': 0.8, 'colsample_bytree': 0.8, 'seed': 974}
0.8230113222002278

LightGBM: 
 {'n_estimators': 1500, 'learning_rate': 0.01, 'num_leaves': 16, 'max_depth': 7, 'subsample': 0.8099999999999999, 'colsample_bytree': 0.71, 'random_state': 734, 'reg_alpha': 0.30000000000000004, 'reg_lambda': 0.30000000000000004, 'min_data_in_leaf': 0}
0.8237743890254952


### 8. Modelling
Building several different models using OneVsRest Classiifer

In [59]:
## Reading the data
train = pd.read_csv('Data/Training.csv')
val = pd.read_csv('Data/Validation.csv')
test = pd.read_csv('Data/Testing.csv')
sub = pd.read_csv('Data/Sample_Submission.csv')

## Defining the input and target variables
X_train = train.drop(columns = ['interest_level']); Y_train = train['interest_level']
X_validation = val.drop(columns = ['interest_level']); Y_validation = val['interest_level']
X_test = test.drop(columns = ['ID'])

In [61]:
## Building the multi-classifier (using RF)
one_vs_all_rf = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 12, min_samples_split = 1, 
                                                                       min_samples_leaf = 1, random_state = 329)).fit(X_train, Y_train)

## Predicting on the test
train_preds = one_vs_all_rf.predict_proba(X_train)
val_preds = one_vs_all_rf.predict_proba(X_validation)
test_preds = one_vs_all_rf.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
val_preds = pd.DataFrame(val_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
val_error = log_loss(Y_validation, val_preds)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

## Creating submission file
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/1vall_rf.csv', index = False)

Training Error: 0.644933124773493
Validation Error: 0.8663288280511712


In [65]:
## Building the multi-classifier (using LGBM)
one_vs_all_lgbm = OneVsRestClassifier(estimator = LGBMClassifier(n_estimators = 1500, learning_rate = 0.01, num_leaves = 16,
                                                                 max_depth = 7, subsample = 0.81, colsample_bytree = 0.71, 
                                                                 random_state = 734, reg_alpha = 0.3, reg_lambda = 0.3, 
                                                                 min_data_in_leaf = 0, verbosity = -1)).fit(X_train, Y_train)
## Predicting on the test
train_preds = one_vs_all_lgbm.predict_proba(X_train)
val_preds = one_vs_all_lgbm.predict_proba(X_validation)
test_preds = one_vs_all_lgbm.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
val_preds = pd.DataFrame(val_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
val_error = log_loss(Y_validation, val_preds)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

## Creating submission file
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/1vall_lgbm.csv', index = False)

[LightGBM] [Warning] min_data_in_leaf is set=0, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=0
Training Error: 0.6621823192997736
Validation Error: 0.8273536739073135


In [63]:
## Building the multi-classifier (using XGB)
one_vs_all_xgb = OneVsRestClassifier(estimator = XGBClassifier(n_estimators = 200, learning_rate = 0.22, max_depth = 3, 
                                                              gamma = 0, min_child_weight = 0, subsample = 0.8, 
                                                              colsample_bytree = 0.8, seed = 974)).fit(X_train, Y_train)

## Predicting on the test
train_pred = one_vs_all_xgb.predict_proba(X_train)
val_pred = one_vs_all_xgb.predict_proba(X_validation)
test_pred = one_vs_all_xgb.predict_proba(X_test)

## Predictions to data-frames
train_pred = pd.DataFrame(train_pred, columns = ['high', 'low', 'medium'])
val_pred = pd.DataFrame(val_pred, columns = ['high', 'low', 'medium'])
test_pred = pd.DataFrame(test_pred, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
val_error = log_loss(Y_validation, val_preds)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

## Creating submission file
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/1vall_xgb.csv', index = False)

Training Error: 0.6621823192997736
Validation Error: 0.8273536739073135


In [64]:
## Building the multi-classifier (using Hist)
one_vs_all_hist = OneVsRestClassifier(estimator = HistGradientBoostingClassifier(learning_rate = 0.02, max_iter = 300, max_depth = 12,
                                                                                l2_regularization = 0.33, random_state = 271)).fit(X_train, Y_train)

## Predicting on the test
train_preds = one_vs_all_hist.predict_proba(X_train)
val_preds = one_vs_all_hist.predict_proba(X_validation)
test_preds = one_vs_all_hist.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
val_preds = pd.DataFrame(val_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
val_error = log_loss(Y_validation, val_preds)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

## Creating submission file
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/1vall_hist.csv', index = False)

Training Error: 0.6697910658686838
Validation Error: 0.8306783179229262


### 9. Principal Component Analysis Work

In [ ]:
train = pd.read_csv('Data/Training.csv')

In [ ]:
pca = PCA().fit(train)

print(pca.explained_variance_ratio_)

In [ ]:
X_pca = pca.components_

X_pca.head()

### 10. Constructing Neural Networks

In [ ]:
"""
Optuna example that optimizes multi-layer perceptrons using Tensorflow (Eager Execution).
In this example, we optimize the validation accuracy of hand-written digit recognition using
Tensorflow and MNIST. We optimize the neural network architecture as well as the optimizer
configuration.
"""

import tensorflow as tf


N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 3
EPOCHS = 10


def create_model(trial):
    n_layers = trial.suggest_int(n_layers, 1, 3)
    weight_decay = trial.suggest_float('weight_decay', 1e-10, 1e-3, log = True)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for i in range(n_layers):
        num_hidden = trial.suggest_int('n_units_l{}'.format(i), 4, 128, log = True)
        model.add(tf.keras.layers.Dense(
            num_hidden,
            activation = 'relu',
            kernel_regularizer = tf.keras.regularizers.l2(weight_decay),))
    model.add(tf.keras.layers.Dense(CLASSES, kernel_regularizer = tf.keras.regularizers.l2(weight_decay)))
    return model


def create_optimizer(trial):
    # We optimize the choice of optimizers as well as their parameters.
    kwargs = {}
    optimizer_options = ['RMSprop', 'Adam', 'SGD']
    optimizer_selected = trial.suggest_categorical('optimizer', optimizer_options)
    if optimizer_selected == 'RMSprop':
        kwargs['learning_rate'] = trial.suggest_float('rmsprop_learning_rate', 1e-5, 1e-1, log = True)
        kwargs['weight_decay'] = trial.suggest_float('rmsprop_weight_decay', 0.85, 0.99)
        kwargs['momentum'] = trial.suggest_float('rmsprop_momentum', 1e-5, 1e-1, log = True)
    elif optimizer_selected == 'Adam':
        kwargs['learning_rate'] = trial.suggest_float('adam_learning_rate', 1e-5, 1e-1, log = True)
    elif optimizer_selected == 'SGD':
        kwargs['learning_rate'] = trial.suggest_float('sgd_opt_learning_rate', 1e-5, 1e-1, log = True)
        kwargs['momentum'] = trial.suggest_float('sgd_opt_momentum', 1e-5, 1e-1, log = True)

    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer


def learn(model, optimizer, dataset, mode = 'eval'):
    accuracy = tf.metrics.Accuracy('accuracy', dtype = tf.float32)

    for batch, (images, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(images, training=(mode == "train"))
            loss_value = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            )
            if mode == "eval":
                accuracy(
                    tf.argmax(logits, axis=1, output_type=tf.int64), tf.cast(labels, tf.int64)
                )
            else:
                grads = tape.gradient(loss_value, model.variables)
                optimizer.apply_gradients(zip(grads, model.variables))

    if mode == "eval":
        return accuracy


def get_mnist():
    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()
    x_train = x_train.astype("float32") / 255
    x_valid = x_valid.astype("float32") / 255

    y_train = y_train.astype("int32")
    y_valid = y_valid.astype("int32")

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(60000).batch(BATCHSIZE).take(N_TRAIN_EXAMPLES)

    valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
    valid_ds = valid_ds.shuffle(10000).batch(BATCHSIZE).take(N_VALID_EXAMPLES)
    return train_ds, valid_ds


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    
    
    train_ds, valid_ds = get_mnist()

    # Build model and optimizer.
    model = create_model(trial)
    optimizer = create_optimizer(trial)

    # Training and validating cycle.
    with tf.device("/cpu:0"):
        for _ in range(EPOCHS):
            learn(model, optimizer, train_ds, "train")

        accuracy = learn(model, optimizer, valid_ds, "eval")

    # Return last validation accuracy.
    return accuracy.result()


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
## Reading the data
train = pd.read_csv('Data/Training.csv')
val = pd.read_csv('Data/Validation.csv')

## Defining the input and target variables
X_train = train.drop(columns = ['interest_level']); Y_train = train['interest_level']
X_validation = val.drop(columns = ['interest_level']); Y_validation = val['interest_level']

md1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])

md1.compile(optimizer = 'sgd', loss = 'log_loss', metrics = ['accuracy'])

## Fitting the model and using the model to predict on the test dataset
model1 = md1.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 3), epochs = 100, batch_size = 500,
               validation_data = (X_validation, tf.keras.utils.to_categorical(Y_validation, num_classes = 3)), verbose = 0)


## Reporting the recall score of this model. Using 15% as the cut-off value
md1_train_preds = md1.predict_proba(X_train)
md1_val_preds = md1.predict_proba(X_validation)

print(log_loss(Y_train, md1_train_preds))
print(log_loss(Y_validation, md1_val_preds))